# Simulation de diffusion thermique 2D

## Équation de diffusion thermique

L'équation de diffusion thermique, aussi connue sous le nom d'équation de la chaleur, est une équation aux dérivées partielles qui décrit comment la température évolue dans un matériau au fil du temps. En 2D, cette équation s'écrit :

$$\frac{\partial T}{\partial t} = D \left( \frac{\partial^2 T}{\partial x^2} + \frac{\partial^2 T}{\partial y^2} \right)$$

où :
- $T(x,y,t)$ est la température au point $(x,y)$ au temps $t$
- $D$ est le coefficient de diffusion thermique (constante positive)
- $\frac{\partial T}{\partial t}$ est la dérivée partielle de la température par rapport au temps
- $\frac{\partial^2 T}{\partial x^2}$ et $\frac{\partial^2 T}{\partial y^2}$ sont les dérivées partielles secondes par rapport à l'espace

## Discrétisation numérique

Pour résoudre cette équation numériquement, on utilise une méthode des différences finies. Les dérivées partielles sont approximées par :

$$\frac{\partial^2 T}{\partial x^2} \approx \frac{T_{i+1,j} - 2T_{i,j} + T_{i-1,j}}{(\Delta x)^2}$$
$$\frac{\partial^2 T}{\partial y^2} \approx \frac{T_{i,j+1} - 2T_{i,j} + T_{i,j-1}}{(\Delta y)^2}$$

L'évolution temporelle est calculée avec la méthode d'Euler explicite :

$$T_{i,j}^{n+1} = T_{i,j}^{n} + D \Delta t \left( \frac{T_{i+1,j}^{n} - 2T_{i,j}^{n} + T_{i-1,j}^{n}}{(\Delta x)^2} + \frac{T_{i,j+1}^{n} - 2T_{i,j}^{n} + T_{i,j-1}^{n}}{(\Delta y)^2} \right)$$

où $n$ indique le pas de temps et $(i,j)$ indique la position spatiale.

In [ ]:
# Importation des bibliothèques nécessaires
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import Video
import os

In [ ]:
# Paramètres de la simulation
nx, ny = 100, 100  # Nombre de points dans la grille
D = 1.0           # Coefficient de diffusion
dx, dy = 1.0, 1.0  # Pas d'espace
dt = 0.1          # Pas de temps
steps = 1000      # Nombre d'étapes de simulation
save_every = 5    # Sauvegarder une image tous les 'save_every' pas de temps

# Création de la grille de température initiale
T = np.zeros((nx, ny))

# Condition initiale : source de chaleur au centre
radius = 5
center_x, center_y = nx // 2, ny // 2
for i in range(nx):
    for j in range(ny):
        if (i - center_x)**2 + (j - center_y)**2 < radius**2:
            T[i, j] = 100.0  # Point chaud

# Ajout d'autres sources de chaleur
radius_small = 3
for cx, cy in [(nx//4, ny//4), (3*nx//4, ny//4), (nx//4, 3*ny//4), (3*nx//4, 3*ny//4)]:
    for i in range(nx):
        for j in range(ny):
            if (i - cx)**2 + (j - cy)**2 < radius_small**2:
                T[i, j] = 50.0  # Sources secondaires

In [ ]:
# Fonction pour calculer un pas de temps de diffusion thermique
def diffusion_step(T, D, dt, dx, dy):
    # Copie de T pour éviter de modifier les valeurs pendant le calcul
    T_new = T.copy()
    
    # Calcul des dérivées secondes par différences finies
    # Pour chaque point intérieur (pas sur les bords)
    for i in range(1, nx-1):
        for j in range(1, ny-1):
            # Dérivée seconde en x
            d2Tdx2 = (T[i+1, j] - 2*T[i, j] + T[i-1, j]) / (dx**2)
            # Dérivée seconde en y
            d2Tdy2 = (T[i, j+1] - 2*T[i, j] + T[i, j-1]) / (dy**2)
            
            # Mise à jour de la température selon l'équation de diffusion
            T_new[i, j] = T[i, j] + D * dt * (d2Tdx2 + d2Tdy2)
    
    return T_new

In [ ]:
# Configuration de la visualisation
fig, ax = plt.subplots(figsize=(10, 8))
plt.title('Simulation de diffusion thermique 2D')

# Premier affichage
im = plt.imshow(T, cmap='inferno', animated=True, vmin=0, vmax=100)
plt.colorbar(im, label='Température')

# Dossier pour stocker les images temporaires
temp_dir = 'temp_frames'
if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)

# Simulation et sauvegarde des images
print("Exécution de la simulation...")
frames = []
for step in range(steps):
    T = diffusion_step(T, D, dt, dx, dy)
    
    # Sauvegarde d'une image à intervalle régulier
    if step % save_every == 0:
        frame_path = f"{temp_dir}/frame_{step//save_every:04d}.png"
        im.set_array(T)
        plt.savefig(frame_path)
        frames.append(frame_path)
    
    # Affichage de la progression
    if step % 100 == 0:
        print(f"Étape {step}/{steps}")

In [ ]:
# Création de la vidéo à partir des images
import subprocess

video_file = 'thermal_diffusion.mp4'
fps = 30

# Commande ffmpeg pour créer la vidéo
cmd = f"ffmpeg -y -framerate {fps} -i {temp_dir}/frame_%04d.png -c:v libx264 -pix_fmt yuv420p -crf 23 {video_file}"
print("Création de la vidéo...")
subprocess.run(cmd, shell=True)
print(f"Vidéo créée: {video_file}")

In [ ]:
# Afficher la vidéo dans le notebook
Video(video_file)

In [ ]:
# Nettoyage des fichiers temporaires (optionnel)
import shutil

# Décommenter pour supprimer les images temporaires
# shutil.rmtree(temp_dir)